In [1]:
from classes.rocket import Rocket

rocket = Rocket("Falcon")
rocket.receive_transmissions()

Received: Message 1
Received: Message 2
Received: Message 3
End of transmission


## Error handling is boring but this is the Exception

### Vito Minheere

# What is an Exception?

In [ ]:
raise "Exception"

An Exception is an object as many other things in Python. Exception extends the BaseException.

This means it can be extended to create other Exceptions with the same attributes

Internal Python Exceptions should use BaseException, all others should use Exception



BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AssertionError
      ├── AttributeError
      ├── EOFError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError

Built-in errors also have their hierarchy. Exception hierarchy can serve an important purpose: 
    When you create such a hierarchy, the user does not have to know all the specific exceptions 

In [12]:
try:
    1 / 0
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e


Arithmetic Error raised


ZeroDivisionError: division by zero

Arithmetic error is higher on the hierarchy than ZeroDivisionError


In [13]:
try:
    1 / 0
except Exception as e:
    print("Exception raised")
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e

Exception raised


except Exception is even higher and will be executed first


In [11]:
try:
    thisssss_function_does_not_exist()
except Exception as e:
    print("Exception raised")
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e

Exception raised



Be aware that Exception will also catch all other Exceptions derived from it

In [6]:
import sys
try:
    sys.exit()
except Exception as e:
    print("SystemExit caught")
except BaseException as e:
    print("SystemExit caught by BaseException")    
except SystemExit as e:
    print("Raised SystemExit")

SystemExit caught by BaseException


SystemExit is above Exception but extends BaseException

# NAASA, NASA As A Service


#### Launch a rocket via a Python module. 
#### Rocket will be supplied. However we have BYOC - Bring Your Own Crew

### No guarantees, only Exceptions

It is rocket science so a lot of things can go wrong. 
Before the launch you could exit your program and call it off if critical errors arise but you can’t just reset the rocket when it is in mid flight. 
We will use a bit of defensive programming to make sure edge cases will be handled and the rocket will not crash. 
In this case we are both the creator of the module and also the user.

## The Rocket

In [ ]:
class Rocket:
    def __init__(self, name):
        self.name = name
        self.altitude = 0
        self.crew = []
        self.fuel_tank = FuelTank("Hydrogen", 100)
        self.command_centre = CommandCentre()
        self.engine = Engine(motors=4)
        self.startup_completed = False
        
    def launch(self):
        if self.startup_completed and self.fuel_tank.volume > 0:
            print(f"{self.name} launched!")
            self.altitude += 100
            self.fuel_tank.volume -= 1

In [ ]:
class FuelTank:
    def __init__(self, fuel_type: str, max_volume: int) -> None:
        self.fuel_type = fuel_type
        self.volume = 0
        self.max_volume = max_volume

# Custom Exceptions

In [10]:
class FuelTypeException(Exception):
    def __init__(self, fuel_type):
        message = f"Invalid fuel type: {fuel_type}. Only 'Hydrogen' fuel is allowed."
        super().__init__(message)


In [ ]:
rocket = Rocket("Falcon", 0, 0)
try:
    rocket.refuel(100, "Diesel")
except FuelTypeException as e:
    print(e)
    rocket.refuel(100, "Hydrogen")

In [19]:
class FuelException(Exception):
    def __str__(self):
        return "Something is wrong in the fuel system"
        
class FuelTypeException(FuelException):
    def __init__(self, fuel_type):
        message = f"Invalid fuel type: {fuel_type}. Only 'Hydrogen' fuel is allowed."
        super().__init__(message)

In [13]:
rocket = Rocket("Falcon", 0, 0)
try:
    rocket.refuel(100, "Diesel")
except FuelException as e:
    print(e)
except FuelTypeException as e:
    print(e)
    rocket.refuel(100, "Hydrogen")

Something is wrong in the fuel system


## Explicit chained exceptions

In [14]:
class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e

crew = ['John', 'Mary', 'Mike']
print('Final check procedure')

personnel_check()


Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike


RocketNotReadyError: Crew is incomplete

In [15]:
try:
    personnel_check()
except RocketNotReadyError as f:
    print('General exception: "{}", caused by "{}"'.format(f, f.__cause__))

	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
General exception: "Crew is incomplete", caused by "list index out of range"


# Suppressing errors

In [16]:
# What if I don't care about an error? Can I just skip them?
inputs = [1, 0, "two", 3, 4]

def divide(a, b):
    return a/b

for value in inputs:
    try:
        print(divide(1, value))
    except (ZeroDivisionError, TypeError) as e:
        pass

1.0
0.3333333333333333
0.25


In [17]:
from contextlib import suppress

inputs = [1, 0, "two", 3, 4]

def divide(a, b):
    return a/b

for value in inputs:
    with suppress((ZeroDivisionError, TypeError)):
        print(divide(1, value))

1.0
0.3333333333333333
0.25


In [ ]:
# give example with Rocket code

# Low fuel error

# Restart engine function
# supress low fuel level error and try the restart anyway